# Méthode Hartree-Fock : un exemple sur des fonctions 1s avec une base de type STO-3G

Ce code sous python vise à montrer sur un exemple simple comment peut se faire un calcul de type Hartree-Fock restreint sur deux exemples : la molécule de dihydrogène et HeH$^+$. Le but est ici d'illustrer la partie algorithmique dans une version simplifiée. Il a initialement été réalisé pour les étudiants suivant l'UE de Modélisation Moléculaire en L3 à l'ENS de Lyon au sein de la formation Sciences de la matière.

Il cherche à venir en complément du cours sur la méthode Hartree-Fock disponible ici :
http://agregationchimie.free.fr/cours.php#HartreeFock

Par souci d'universalité, les équations font références au livre :

* **Modern quantum chemistry : introduction to advanced electronic structure theory**
  *Attila Szabo, Neil S. Ostlund*
  ISBN 9780486691862


Ce code s'inspire lourdement de cette page :
https://medium.com/analytics-vidhya/practical-introduction-to-hartree-fock-448fc64c107b

Ce code est mis à disposition sous licence CC-BY-NC-SA.


* [Lecture d'un fichier xyz](#xyz)
* [Lecture des fichiers de base](#basis-set)
* [Nombre d'électrons](#nb-elec)
* [Calcul des intégrales](#calcul-int)
 * [Définition d'une classe qui correspond à une orbitale](#classe-orb)
 * [Définition d'une classe qui correspond à un produit d'orbitale](#classe-prod)
 * [Création de la base des orbitales atomiques $\chi_\mu$](#orb-creation)
 * [Calcul du recouvrement](#overlap)
 * [Calcul de l'énergie cinétique](#kinetic)
 * [Calcul de l'énergie potentielle électron-noyau](#TNe)
 * [Calcul des intégrales bi-électroniques](#intbi)
* [Initialisation de la matrice densité/des coefficients](#densitymatrix)
* [Construction de la partie bi-électronique](#intbiG)
* [Procédure SCF](#scf)

In [1]:
#import de librairies
import numpy as np
import scipy
from scipy.special import erf
import json

Définition de quelques variables usuelles : abbréviations des éléments et leur numéro atomique. En pratique, ici, on se limitera à H et He.

In [2]:
#Atomes 
AtomList = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og']
AtomCharges = np.arange(1,119,1)

<span id="xyz"></span>
# Lecture d'un fichier xyz

Les fichiers .xyz sont un format courant d'import export de coordonnées atomiques. Cette fonction va donc permettre de les lire automatiquement. Dans un fichier .xyz La première ligne indique combien d'atomes sont présents, la deuxième ligne est une ligne de commentaire. Les lignes suivantes indiquent le type d'atome puis leurs coordonnées selon x y et z.



In [3]:
def read_xyz_file(file):
    """
    Fonction pour lire un fichier xyz
    - file est le nom du fichier à lire qui doit être au format xyz
    """
    file = open(file,'r')
    nbAtoms = 0 #nombre d'atomes lus ligne à ligne
    check = 0 # nombre d'atomes indiqués sur la première ligne
    Atoms = [] # Liste des types d'atomes
    coordinates = [] #Liste des coordonnées des atomes
    #lecture du fichier ligne à ligne
    for idx, line in enumerate(file):
        #Pour la première ligne du fichier, on regarde combien d'atomes on est censé lire
        if idx == 0:
            check = int(line.split()[0])
        #On ignore la ligne de commentaire puis on lit les coordonnées ligne à ligne
        elif idx>1:
            split = line.split() #découpage des lignes
            Atoms.append(split[0]) #lecture du type d'atome
            #lecture des coordonnées et conversion en type float
            coordinates.append(np.array([split[1],split[2],split[3]],dtype='float'))
            nbAtoms+=1 #on incrémente le type d'atomes 
    file.close() #clôture du fichier
    if check != nbAtoms:
        print('Attention, le nombre de coordonnées lues est différent du nombre théorique d\'atome dans la molécule')
    return nbAtoms, Atoms, coordinates

In [4]:
#fichier d'exemple
file = 'H2.xyz'
file = 'HeH.xyz'
#lecture du fichier .xyz avec les coordonnées des atomes
nbAtoms, Atoms, coordinates = read_xyz_file(file)
"""
On créé un nouveau tableau avec les charges des atomes pour ensuite pouvoir consruire 
le potentiel d'interaction électron-noyau T_Ne. Pour cela, on cherche la position de la chaîne de caractère 
correspondant au symbole atomique puis on fait la correspondance grâce au tableau AtomCharges
"""
AtomicCharges = []
for atom in Atoms:
    idx = AtomList.index(atom)
    AtomicCharges.append(AtomCharges[idx])

#Affichage pour contrôler les variables lues et la correspondance des charges    
#print(AtomicCharges)
#print(nbAtoms)
#print(Atoms)
#print(coordinates)

<span id="basis-set"></span>
# Lecture des fonctions de base

On va maintenant définir les fonctions de base, en utilisant la base minimale de type STO-3G.

Le fichier obtenu provient du site https://www.basissetexchange.org/ avec un export au format json.

In [5]:
def readBasisSetJSON(file):
    """
    file doit contenir les fonctions de base au format JSON exporté à partir du site https://www.basissetexchange.org/
    ATTENTION :
    - les fonctions de bases pour l'hydrogène du Szabo (p159) correspondent à la version 0 du site, 
      il faut faire un export avec le mode "advanced". 
    - De même, les fonctions de base utilisées pour l'hélium sont celles du site et les valeurs diffèrent 
      de celles données dans le Szabo p170 
    - Le programme ne fonctionne PAS avec les orbitales p, donc il n'est PAS capable de faire des calculs pour des
      molécules contenant des atomes autre que l'hélium ou l'hydrogène
    - Pour que la correspondance entre type d'atome et base de fonction soit correct, il faut que le fichier json 
      contienne les bases pour tous les types d'atomes jusqu'au numéro atomique de l'élément le plus lourd
    """ 
    f = open(file_basis)
    basis_sets = json.load(f)
    #Exposants et coefficients pour les orbitales s
    Sexponents = []
    Scoefficients = []
    #Exposants et coefficients pour les orbitales p
    Pexponents = []
    Pcoefficients = []

    #Pour chaque élement, on lit les fonctions de base :
    for i,element in enumerate(basis_sets['elements']):
        #lecture pour chaque couche
        #print(AtomList[int(element)-1]) #affichage de l'élément
        #On ajoute une liste d'exposants et coefficients pour chaque nouvel élément
        Sexponents.append([])
        Pexponents.append([])
        Scoefficients.append([])
        Pcoefficients.append([])
        """
        Ensuite, on lit progressivement par valeur croissante, cela permet de naturellement remplir par couche croissante 
        1s puis 2s, etc. 
        Ne marche plus à partir des atomes ayant des orbitales d (Z>=21, Scandium)
        """
        for shell in basis_sets['elements'][element]['electron_shells']: 
            #lecture des coefficients pour chaque sous-couche
            for subshell in shell['angular_momentum']:
                AtomExponents = np.asarray(shell['exponents'],dtype="float")
                AtomCoefficients = np.asarray(shell['coefficients'][subshell],dtype="float")
                if subshell == 0:
                    Sexponents[i].append(AtomExponents)
                    Scoefficients[i].append(AtomCoefficients)
                if subshell == 1:
                    Pexponents[i].append(AtomExponents)
                    Pcoefficients[i].append(AtomCoefficients)
    return Sexponents,Scoefficients,Pexponents,Pcoefficients

In [6]:
#fichier contenant les fonctions de base
file_basis = 'sto-3g0.json'
Sexponents,Scoefficients,Pexponents,Pcoefficients = readBasisSetJSON(file_basis)

In [7]:
#Affichage des exposants puis des coefficients pour chaque orbitale gaussienne de la contraction pour les orbitales s
for i,coeffs in enumerate(Sexponents):
    print(AtomList[i])
    for j,exponents in enumerate(coeffs):
        print('{}s'.format(j+1))
        print(exponents)
        print(Scoefficients[i][j])

H
1s
[3.42525091 0.62391373 0.1688554 ]
[0.15432897 0.53532814 0.44463454]
He
1s
[6.36242139 1.158923   0.31364979]
[0.15432897 0.53532814 0.44463454]


<span id="nb-elec"></span>
# Nombre d'électrons au sein de la molécule

In [8]:
NbElectrons = int(2)
#nombre d'orbitales doublement occupées (on se place en formalisme restreint)
orbOccupied = int(NbElectrons/2)

<span id="calcul-int"></span>
# Calcul des intégrales

<span id="classe-orb"></span>
### Définition d'une classe qui correspond à une orbitale


ATTENTION :
$\chi^\mathrm{G}_{\alpha_{p\mu},\mathbf{R_A}}\left(\mathbf{r} \right) = \left(\dfrac{2 \alpha_{p\mu}}{\pi}\right)^{0.75}\exp(-\alpha_{p\mu} |\mathbf{r}-\mathbf{r}_A|^2)$

Nous allons avoir besoin de stocker des orbitales. Plutôt que de stocker la fonction complète sous sa forme $\chi^{CGF}_{\mu}\left(\mathbf{r}-\mathbf{R}_A \right) = \sum_{p=1}^{L} c_{p\mu}\chi^\mathrm{G}_{\alpha_{p\mu},\mathbf{R_A}}\left(\mathbf{r} \right)$
on va la stocker sous la forme de deux tableaux :
* un tableau avec les exposants de la gaussienne $\{\alpha_{p\mu}\}$
* un tableau avec les coefficients de la gaussienne $\{c_{p\mu}\}$ 
Lorsque l'on devra faire des produits de gaussienne, il sera alors plus facile de trouver l'expression des gaussiennes correspondant au produit de fonction.

On en profite également pour rendre humainement plus compréhensible l'orbitale dont il est question : pour cela, on donne pour nom à l'orbitale : 
- l'indice de l'atome dans le tableau `Atoms`
- suivi du symbole de l'élemnt
- suivi de la couche s correspondante

Ainsi, pour la molécule HeH$^+$, la première orbitale s'appelera OHe1s et la deuxième 1H1s. 

On note également les coordonnées de l'atome sur lequel est centré l'orbitale gaussienne.



In [9]:
class Orbital:
    """
    name Le nom de l'orbitale : numéro de l'atome, type puis type d'orbitale
    center : coordonnées 3D de l'atome portant l'OA
    exponents :  les exposants des gaussiennes sous forme de tableau numpy
    coeffs : les coefficients associés à chaque primitive
    """
    def __init__(self, name, center, exponents, coeffs):
        self.name = name
        self.center = center
        self.exponents = exponents
        self.coeffs = coeffs
    def __repr__(self):
        return 'name :         {}\ncenter :       {}\nexponents    : {}\ncoefficients : {}'.format(self.name,self.center,self.exponents,self.coeffs)

<span id="classe-prod"></span>
### Définition d'une classe qui correspond à un produit d'orbitale
Idem, on crée une classe qui contient un produit d'orbitales atomiques pour cela, on stocke 
- `p` les exposants des nouvelles gaussiennes qui sont la somme des 2 exposants ($p = \alpha+ \beta$ équation 3.209 du Szabo)
- `ab` qui est le produit des deux exposants (utile pour calculer certaines intégrales par la suite) ($\alpha\times \beta$)
- `diffR` la norme de l'écart entre les deux centres des gaussiennes d'origine ($|\mathbf{r}_A-\mathbf{r}_B|^2$)
- `K` la constante de normalisation correspondant au produit $\mathtt{K}= \exp(-\mathtt{ab}/\mathtt{p}*\mathtt{diffR})\times \left(\dfrac{4 \alpha\times \beta }{\pi^2}\right)^{0.75} \times c_A\times c_B $ (équation 3.208 avec un facteur $\left(\dfrac{4 \alpha\times \beta }{\pi^2}\right)^{0.75}$ supplémentaire pour prendre en compte le facteur de normalisation des gaussiennes et les coefficients $c_A$ et $c_B$ qui sont les $c_{p\mu}$.)
- `Rp` la position du nouveau centre résultant du produit de gaussiennes

On a donc $\chi_\mu\times\chi_\nu = \sum_i K_i\exp(-p_i |\mathbf{r}-\mathbf{Rp}|^2)$

In [10]:
class OrbitalProduct:
    """
    name : nom du produit
    p : exposant associé à chacun des termes du produit
    diffR : norme de l'écart entre les centres atomiques
    K : coefficient normalisé associé à chacun des produits
    Rp : position de l'orbitale associée au produit
    """
    def __init__(self,name,p, ab,diffR, K, Rp):
        self.name = name
        self.p = p
        self.ab = ab
        self.diffR = diffR
        self.K = K
        self.Rp = Rp

On utilise les résultats du Szabo p 154, ou 411 pour effectuer le calcul de la gaussienne correspondant au produit de deux gaussiennes simples.

In [11]:
#calcul d'un produit de gaussienne simple (Szabo p411)
#Le facteur N permet d'avoir des orbitales normalisées pour avoir le bon résultat
def ProductGaussians(orba,orbb,i,j):
    #somme des exposants
    p = orba.exponents[i] + orbb.exponents[j]
    ab = orba.exponents[i]*orbb.exponents[j]
    #calcul de la norme de la différence des centres pour la normalisation
    diffR = (np.linalg.norm(orba.center-orbb.center))**2
    #facteur de normalisation+coefficients
    N = (4*ab/(np.pi**2))**0.75
    K = N*np.exp(-ab/p*diffR)*orba.coeffs[i]*orbb.coeffs[j]
    #centre de la nouvelle orbitale
    Rp = (orba.exponents[i]*orba.center+orbb.exponents[j]*orbb.center)/p
    return p,ab, diffR, K, Rp

Puis on généralise le calcul à des produits d'orbitales qui sont la somme de gaussiennes. Le produit de deux orbitales de type STO-3G donne donc naissance à 9 orbitales gaussiennes.

In [12]:
def ProductOrbitals(orba,orbb):
    ps=[]
    Ks=[]
    Rps=[]
    abss=[]
    for i in range(orba.exponents.shape[0]):
        for j in range(orbb.exponents.shape[0]):
            p, ab, diffR, K, Rp = ProductGaussians(orba,orbb,i,j)
            ps.append(p)
            Ks.append(K)
            Rps.append(Rp)
            abss.append(ab)
    return OrbitalProduct('{}x{}'.format(orba.name,orbb.name),np.asarray(ps),np.asarray(abss),diffR,np.asarray(Ks),np.asarray(Rps))

<span id="orb-creation"></span>
## Création de la base des orbitales atomiques $\chi_\mu$
Ensuite, pour chaque atome, on lui associe les orbitales atomiques correspondantes.

`nbOrbitals` est le nombre total de fonctions de base (noté $M$ dans le polycopié)

In [13]:
Orbitals = []
#Nombre total d'orbitales à considérer
nbOrbitals = 0

"""
Assignation des orbitales pour chaque atome : on attache les orbitales associées à chaque élément
pour chaque atome à sa position
"""
for i,atom in enumerate(Atoms):
    idx = AtomList.index(atom)
    for j,exponents in enumerate(Sexponents[idx]):
        Orbitals.append(Orbital('{}{}{}s'.format(i,atom,j+1),coordinates[i],exponents,Scoefficients[idx][j]))
        nbOrbitals +=1    

In [14]:
#Affichage des orbitales considérées        
for i,orb in enumerate(Orbitals):
    print(orb)

name :         0He1s
center :       [0. 0. 0.]
exponents    : [6.36242139 1.158923   0.31364979]
coefficients : [0.15432897 0.53532814 0.44463454]
name :         1H1s
center :       [0.     0.     1.4632]
exponents    : [3.42525091 0.62391373 0.1688554 ]
coefficients : [0.15432897 0.53532814 0.44463454]


<span id="overlap"></span>
## Calcul du recouvrement
On utilise la formule A.9 p412 du Szabo adaptée avec le `K` prenant en compte les coefficients, la normalisation, etc.

In [15]:
def Overlap(gp):
    """
    Calcul du recouvrement qui est l'intégrale du produit de deux gaussiennes
    La formule commentée est équivalente à celle utilisée.
    - gp est un objet de type `OrbitalProduct`
    """
    #S = 0
    #for j in range(len(gp.p)):
    #    S+=gp.K[j]*(np.pi/gp.p[j])**1.5
    S = np.sum(gp.K*(np.pi/gp.p)**1.5)
    return S

In [16]:
#Initialisation de la matrice de recouvrement
OverlapMatrix = np.zeros((nbOrbitals,nbOrbitals))

#Calcul de chacun des termes de la matrice, S étant diagonal symétrique, on évite de faire deux fois le même calcul.
for i in range(nbOrbitals):
    for j in range(i,nbOrbitals):
        product = ProductOrbitals(Orbitals[i],Orbitals[j])
        S = Overlap(product)
        OverlapMatrix[i,j]=S
        OverlapMatrix[j,i]=S

print(OverlapMatrix)

[[0.99999999 0.53681935]
 [0.53681935 0.99999999]]


<span id="kinetic"></span>
## Calcul de l'énergie cinétique
On utilise la formule A.11 p412 du Szabo.

In [17]:
def Kinetic(gp):
    """
    Calcul de l'énergie cinétique
    La formule commentée est équivalente à celle utilisée.
    - gp est un objet de type `OrbitalProduct`
    """
    #K = 0
    #for j in range(len(gp.p)):
    #    prefactor = gp.ab[j]/gp.p[j]*(3-2*gp.ab[j]/gp.p[j] *gp.diffR )
    #    K+=prefactor*gp.K[j]*(np.pi/gp.p[j])**1.5
    return np.sum(gp.ab/gp.p *(3-2*gp.ab/gp.p*gp.diffR)*gp.K*(np.pi/gp.p)**1.5)   

In [18]:
#Initialisation de la matrice de recouvrement
KineticMatrix = np.zeros((nbOrbitals,nbOrbitals))

#Calcul de chacun des termes de la matrice, K étant diagonal symétrique, on évite de faire deux fois le même calcul.
for i in range(nbOrbitals):
    for j in range(i,nbOrbitals):
        product = ProductOrbitals(Orbitals[i],Orbitals[j])
        K = Kinetic(product)
        KineticMatrix[i,j]=K
        KineticMatrix[j,i]=K

print(KineticMatrix)

[[1.41176317 0.19744319]
 [0.19744319 0.76003188]]


<span id="TNe"></span>
## Calcul de l'énergie potentielle électron-noyau
On utilise la formule A.33 p415 du Szabo. 

In [19]:
#Fonction définie équation A.32 page 415
def Fo(t):
    if t==0:
        return 1
    else:
        return (0.5*(np.pi/t)**0.5)*erf(t**0.5)
    
def Potential(gp):
    """
    Calcul de l'énergie électron-noyau
    La formule commentée est équivalente à celle utilisée.
    - gp est un objet de type `OrbitalProduct`
    """
    V = 0
    for j in range(len(gp.p)):
        #On somme les termes en Z/rAi :
        for idx,Z in enumerate(AtomicCharges):
            prefactor = -2*np.pi/gp.p[j]*Z*Fo(gp.p[j]*(np.linalg.norm(gp.Rp[j]-coordinates[idx]))**2)
            V+=prefactor*gp.K[j]
    return V   

In [20]:
#Initialisation de la matrice d'énergie potentielle
VMatrix = np.zeros((nbOrbitals,nbOrbitals))

#Calcul de chacun des termes de la matrice, V étant diagonal symétrique, on évite de faire deux fois le même calcul.
for i in range(nbOrbitals):
    for j in range(i,nbOrbitals):
        product = ProductOrbitals(Orbitals[i],Orbitals[j])
        V = Potential(product)
        VMatrix[i,j]=V
        VMatrix[j,i]=V

print(VMatrix)

[[-4.01004617 -1.62927168]
 [-1.62927168 -2.49185753]]


## Calcul de la partie mono-électronique $H^\mathrm{core}$
équation 3.153 p141 du Szabo

In [21]:
HcoreMatrix = KineticMatrix+VMatrix
print(HcoreMatrix)

[[-2.59828299 -1.43182849]
 [-1.43182849 -1.73182566]]


<span id="intbi"></span>
## Calcul des intégrales bi-électroniques
On utilise la formule A.41 p 416 du Szabo

In [22]:
def integralbi(orba,orbb,orbc,orbd):
    #a et b sont occupées par l'électron 1
    #c et d sont occupées par l'électton 2
    prAB = ProductOrbitals(orba,orbb)
    prCD = ProductOrbitals(orbc,orbd)
    I=0
    for i in range(len(prAB.p)):
        for j in range(len(prCD.p)):
            prefactor = (2*np.pi**2.5)/(prAB.p[i]*prCD.p[j]*(prAB.p[i]+prCD.p[j])**0.5)*Fo(prAB.p[i]*prCD.p[j]/(prAB.p[i]+prCD.p[j])*(np.linalg.norm(prAB.Rp[i]-prCD.Rp[j]))**2)
            I+=prefactor*prAB.K[i]*prCD.K[j]
    return I 

In [23]:
"""
Vérification des valeurs avec les valeurs du Szabo p162 equation 3.235
combi = (0,1,0,1)
print(combi)
print(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))
combi = (0,0,1,1)
print(combi)
print(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))
combi = (0,0,0,0)
print(combi)
print(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))
combi = (1,1,1,1)
print(combi)
print(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))
"""

'\nVérification des valeurs avec les valeurs du Szabo p162 equation 3.235\ncombi = (0,1,0,1)\nprint(combi)\nprint(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))\ncombi = (0,0,1,1)\nprint(combi)\nprint(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))\ncombi = (0,0,0,0)\nprint(combi)\nprint(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))\ncombi = (1,1,1,1)\nprint(combi)\nprint(integralbi(Orbitals[combi[0]],Orbitals[combi[1]],Orbitals[combi[2]],Orbitals[combi[3]]))\n'

 <span id="densitymatrix"></span>
 # Initialisation de la matrice densité/des coefficients

In [24]:
def diagonalizeAscendingEigenvalues(M):
    """
    Diagonalise la matrice M en triant les valeurs propres par ordre croissant.
    """
    eigVal,eigVec = np.linalg.eig(M)
    idx = np.argsort(eigVal)
    eigVal = eigVal[idx]
    eigVec = eigVec[:,idx]
    return eigVal,eigVec

In [25]:
def buildPmatrix(Cmatrix,nbOrbitals,orbOccupied):
    """
    Construction de la matrice densité à partir 
    - des coefficients de Cmatrix
    - des orbitales au nombre de nbOrbitals
    - orbOccupied qui est le nombre d'orbitales moléculaires **occupées**.
    équation 3.145 du Szabo
    """
    Pmatrix = np.zeros_like(Cmatrix)
    for i in range(nbOrbitals):
        for j in range(nbOrbitals):
            Pmatrix[i][j]=2*np.sum(Cmatrix[i,0:orbOccupied]*Cmatrix[j,0:orbOccupied])
            Pmatrix[j][i]=Pmatrix[i][j]
    return Pmatrix

Pour la matrice densité initiale, on commence ici par diagonaliser la partie mono-électronique.

In [26]:
E0,C0 = diagonalizeAscendingEigenvalues(HcoreMatrix)
Cmatrix = C0
print(Cmatrix)

#Construction de la matrice densité à partir des coefficients
Pmatrix = buildPmatrix(Cmatrix,nbOrbitals,orbOccupied)

[[-0.80299565  0.59598488]
 [-0.59598488 -0.80299565]]


<span id="intbiG"></span>
# Construction de la partie bi-électronique
équation 3.154 p141 du Szabo

In [27]:
def BuildBiMatrix(nbOrbitals,Pmatrix,Orbitals):
    """
    integralbi utilise la notation "chimiste"
    - nbOrbitals nombre d'orbitales 
    - Pmatrix matrice densité
    - Orbitals orbitales atomiques
    """
    BiMatrix = np.zeros((nbOrbitals,nbOrbitals))
    for mu in range(nbOrbitals):
        for nu in range(nbOrbitals):
            G = 0
            for sigma in range(nbOrbitals):
                for llambda in range(nbOrbitals):
                    G += Pmatrix[sigma][llambda]*(integralbi(Orbitals[mu],Orbitals[nu],Orbitals[sigma],Orbitals[llambda])-0.5*integralbi(Orbitals[mu],Orbitals[llambda],Orbitals[sigma],Orbitals[nu]))
            BiMatrix[mu,nu]=G
            BiMatrix[nu,mu]=G
    return BiMatrix

<span id="symmbase"></span>
# Orthogonalisation de la base des orbitales atomiques
Voir p 142-143 du Szabo.

In [28]:
#Diagonalisation de la matrice de recouvrement; formule 3.166 du Szabo.
evalS,U = np.linalg.eig(OverlapMatrix)
"""
U est la matrice de passage telle que S' soit diagonale si on fait 
U.T × OverlapMatrix × U = s avec s diagonale
"""
print(U)
#On prend les valeurs propres, on les place sur la diagonale puis on prend s^-0.5
invsqrtS = np.diag(evalS**-0.5)
#équation 3.167 du Szabo
matZ = np.dot(U,np.dot(invsqrtS,U.T))
print(matZ)
#vérification de l'équation (3.168) comme quoi la base est orthonormée avec matZ
#print(np.dot(matZ.T,np.dot(OverlapMatrix,matZ)))

[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]
[[ 1.13800243 -0.33134602]
 [-0.33134602  1.13800243]]


<span id="scf"></span>
# Procédure SCF

## Critère de convergence
On regarde si la matrice densité a convergé par rapport à l'itération précédente. On pourrait prendre un critère sur l'énergie ou les deux.

In [29]:
def diffP(P1,P2):
    """
    - P1 matrice densité à l'étape précédente
    - P2 matrice densité obtenue à cette étape
    """
    return np.sqrt(np.sum((P1-P2)**2))

## Procédure SCF
Les étapes sont celles du Szabo p 146

In [30]:
# Nombre d'itérations, on arrête si le calcul ne converge pas
step = 1
# On initialise la valeur du critère de convergence à une valeur élevée pour amorcer la boucle
Threshold = 100
while Threshold > 1e-6 and step < 100:
    #Construction de la partie bi-électronique (étape 5)
    BiMatrix = BuildBiMatrix(nbOrbitals,Pmatrix,Orbitals)
    #print(BiMatrix)
    #Construction de l'opérateur de Fock (étape 6)
    Fmatrix = HcoreMatrix + BiMatrix
    #Construction de la matrice de Fock F' dans la base d'orbitales orthogonales (étape 7)
    FmatrixPrime = np.dot(matZ.T,np.dot(Fmatrix,matZ))
    #Diagonalisation de F' (étape 8)
    E,Cprime = diagonalizeAscendingEigenvalues(FmatrixPrime)
    #Calcul des coefficients dans la base de départ (étape 9)
    Cmatrix = np.dot(matZ,Cprime)
    #Calcul de l'énergie totale (eq 3.274)
    Etot = 0.5*np.sum(Pmatrix*(HcoreMatrix+Fmatrix))
    
    #Stockage de l'ancienne matrice densité
    oldP = Pmatrix.copy()
    #calcul de la nouvelle matrice densité (étape 10)
    Pmatrix = buildPmatrix(Cmatrix,nbOrbitals,orbOccupied)
    #calcul du critère de convergence (étape 11)
    Threshold = diffP(oldP,Pmatrix)

    print('F')
    print(Fmatrix)
    print('S')
    print(OverlapMatrix)
    print('Énergies')
    print(E)
    print('C')
    print(Cmatrix)
    print('P')
    print(Pmatrix)
    print('E')
    print(Etot)    
    print('step')
    print(step)
    step+=1


F
[[-1.15258827 -0.9757405 ]
 [-0.9757405  -0.48775592]]
S
[[0.99999999 0.53681935]
 [0.53681935 0.99999999]]
Énergies
[-1.26552808  0.43280733]
C
[[-0.80209877  0.87262665]
 [-0.30565053 -1.14517137]]
P
[[1.28672487 0.49032383]
 [0.49032383 0.1868445 ]]
E
-5.511356595936597
step
1
F
[[-1.61195766 -1.09173359]
 [-1.09173359 -0.86342246]]
S
[[0.99999999 0.53681935]
 [0.53681935 0.99999999]]
Énergies
[-1.66186212 -0.168999  ]
C
[[-0.86681105  0.80838005]
 [-0.21670705 -1.16528012]]
P
[[1.50272281 0.37568813]
 [0.37568813 0.09392389]]
E
-4.188527237452284
step
2
F
[[-1.59331222 -1.06556452]
 [-1.06556452 -0.83814231]]
S
[[0.99999999 0.53681935]
 [0.53681935 0.99999999]]
Énergies
[-1.63699105 -0.17163222]
C
[[-0.87513221  0.79936432]
 [-0.20463355 -1.16746084]]
P
[[1.53171277 0.35816282]
 [0.35816282 0.08374978]]
E
-4.208334471488821
step
3
F
[[-1.59066012 -1.06171815]
 [-1.06171815 -0.83464577]]
S
[[0.99999999 0.53681935]
 [0.53681935 0.99999999]]
Énergies
[-1.63343944 -0.17234742]
C
[[-0